<a href="https://colab.research.google.com/github/Victoria-research/HFT_Activity_Dynamics/blob/main/Nasdaq100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive
import pyarrow.parquet as pq
from pathlib import Path
import os

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
nasdaq_0117= "/content/drive/MyDrive/Nasdaq100_20240117.zip"

### **Nasdaq100 stocks**

In [ ]:
stock_names = [
    'ADBE', 'ADP', 'ABNB', 'GOOGL', 'GOOG', 'AMZN', 'AMD', 'AEP', 'AMGN', 'ADI',
    'ANSS', 'AAPL', 'AMAT', 'ASML', 'AZN', 'TEAM', 'ADSK', 'BKR', 'BIIB', 'BKNG',
    'AVGO', 'CDNS', 'CDW', 'CHTR', 'CTAS', 'CSCO', 'CCEP', 'CTSH', 'CMCSA', 'CEG',
    'CPRT', 'CSGP', 'COST', 'CRWD', 'CSX', 'DDOG', 'DXCM', 'FANG', 'DLTR', 'DASH',
    'EA', 'EXC', 'FAST', 'FTNT', 'GEHC', 'GILD', 'GFS', 'HON', 'IDXX', 'ILMN',
    'INTC', 'INTU', 'ISRG', 'KDP', 'KLAC', 'KHC', 'LRCX', 'LULU', 'MAR', 'MRVL',
    'MELI', 'META', 'MCHP', 'MU', 'MSFT', 'MRNA', 'MDLZ', 'MDB', 'MNST', 'NFLX',
    'NVDA', 'NXPI', 'ORLY', 'ODFL', 'ON', 'PCAR', 'PANW', 'PAYX', 'PYPL', 'PDD',
    'PEP', 'QCOM', 'REGN', 'ROP', 'ROST', 'SIRI', 'SPLK', 'SBUX', 'SNPS', 'TTWO',
    'TMUS', 'TSLA', 'TXN', 'TTD', 'VRSK', 'VRTX', 'WBA', 'WBD', 'WDAY', 'XEL', 'ZS'
]

## **Manipulate ITCH data to quantify HFT activity into HFT Cancellation to Execution Ratio, HFT Share of trades, HFT Volume Share, Hidden Share of trades, Hidden Share of Volume**

In [ ]:
# Column names
column_names = ["Time", "Seq Num", "MPID", "BuySell", "CurrShares", "Price", "DelShares", "MsgType", "PriorSeq", "OrderLife", "TrID"]

# Initialize a dictionary to store results
results_dict = {'Stock': [], 'Cancellation to Execution Ratio': [],'Volume':[],'HFT Share of Trades':[],
                'HFT Cancellation to Execution Ratio': [], 'HFT Volume Share': [],
                'Hidden Share of Trades': [], 'Hidden Share of Volume': []}

# Loop over each stock file
for stock_name in stock_names:
    parquet_file_path = f'{nasdaq_0117}/{stock_name}_20240117_tick.parquet'

    try:
        table = pq.read_table(parquet_file_path)
        df = table.to_pandas()

        # Set column names
        df.columns = column_names

        # Filter data for the specified time range
        itch_data_filter = df.query("Time >= 93500 & Time <= 155500")

        # Calculate metrics
        nA = itch_data_filter['MsgType'].str.count('A').sum()
        nC = itch_data_filter['MsgType'].str.count('C').sum()
        nD = itch_data_filter['MsgType'].str.count('D').sum()
        nE = itch_data_filter['MsgType'].str.count('E').sum()
        nF = itch_data_filter['MsgType'].str.count('F').sum()
        nP = itch_data_filter['MsgType'].str.count('P').sum()
        nU = itch_data_filter['MsgType'].str.count('U').sum()
        nX = itch_data_filter['MsgType'].str.count('X').sum()

        cancellation_to_execution_ratio = (nD + nU + nX) / (nC + nE)

        itch_data_hft = itch_data_filter.query("OrderLife < 0.050 and MsgType!='A' and MsgType!='F'")
        nCHFT = itch_data_hft['MsgType'].str.count('C').sum()
        nDHFT = itch_data_hft['MsgType'].str.count('D').sum()
        nEHFT = itch_data_hft['MsgType'].str.count('E').sum()
        nUHFT = itch_data_hft['MsgType'].str.count('U').sum()
        nXHFT = itch_data_hft['MsgType'].str.count('X').sum()

        hft_cancellation_to_execution_ratio = (nDHFT + nUHFT + nXHFT) / (nCHFT + nEHFT)

        Cdata = itch_data_filter.query("MsgType=='C'")
        Cvolume = Cdata['CurrShares'].sum()

        CHFTdata = itch_data_filter.query("OrderLife < 0.050 and MsgType=='C'")
        CHFTvolume = CHFTdata['CurrShares'].sum()

        Edata = itch_data_filter.query("MsgType=='E'")
        Evolume = Edata['CurrShares'].sum()

        total_volume= Cvolume + Evolume

        EHFTdata = itch_data_filter.query("OrderLife < 0.050 and MsgType=='E'")
        EHFTvolume = EHFTdata['CurrShares'].sum()

        hft_volume_share = 100 * (CHFTvolume + EHFTvolume) / (Cvolume + Evolume)
        hft_share_of_trades = 100 * (nCHFT+nEHFT)/(nC+nE)

        Pdata = itch_data_filter.query("MsgType=='P'")
        Pvolume = Pdata['DelShares'].sum()

        hidden_share_of_trades = 100 * nP / (nC + nE + nP)
        hidden_share_of_volume = 100 * Pvolume / (Cvolume + Evolume + Pvolume)

        # Append results to the dictionary
        results_dict['Stock'].append(stock_name)
        results_dict['Cancellation to Execution Ratio'].append(cancellation_to_execution_ratio)
        results_dict['Volume'].append(total_volume)
        results_dict['HFT Cancellation to Execution Ratio'].append(hft_cancellation_to_execution_ratio)
        results_dict['HFT Share of Trades'].append(hft_share_of_trades)
        results_dict['HFT Volume Share'].append(hft_volume_share)
        results_dict['Hidden Share of Trades'].append(hidden_share_of_trades)
        results_dict['Hidden Share of Volume'].append(hidden_share_of_volume)

    except FileNotFoundError:
        print(f'File not found: {parquet_file_path}')

# Create a DataFrame from the results dictionary
results_df = pd.DataFrame(results_dict)

# Save the DataFrame to an Excel file
excel_file_path = '/content/drive/MyDrive/Nasdaq100_20240117_Results.xlsx'
results_df.to_excel(excel_file_path, index=False)

# Print the results DataFrame
print("Results DataFrame:")
print(results_df)

# Print the path to the saved Excel file
print(f"\nResults saved to: {excel_file_path}")
